In [1]:
import sys
sys.path.append('gorani.zip')
sc.addPyFile('gorani.zip')
from gorani import firebase
firebase.init('spark')
mydb = firebase.db()
from gorani.gorani import Gorani
from gorani.transformer import Transformer
from gorani.utils import split_sentence
gorani = Gorani(mydb)
transformer = Transformer(gorani, spark, sc)

In [2]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [3]:
import os
df = spark.read.csv("logs.csv", header="true", quote='"', escape='"')

In [18]:
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql.functions import udf, from_json
import pyspark.sql.functions as F
from pyspark.sql.types import *

PaginateSentence = StructType([
    StructField("sid", StringType()),
    StructField("words", ArrayType(StringType())),
    StructField("wordUnknowns", ArrayType(IntegerType())),
    StructField("unknown", BooleanType())
])

SubmitQuestionPayload = StructType([
    StructField("bookId", StringType()),
    StructField("chapterId", StringType()),
    StructField("questionId", StringType()),
    StructField("option", StringType()),
    StructField("right", BooleanType()),
    StructField("time", IntegerType())
])


@udf(ArrayType(PaginateSentence))
def get_sentences(bookId, chapterId, sids, wordUnknowns, sentenceUnknowns):
    out = []
    for sid in sids:
        wus = []
        unknown = False
        sentence = gorani.get_sentence(bookId, chapterId, sid)
        words = split_sentence(sentence)
        for su in sentenceUnknowns:
            if su['sentenceId'] == sid:
                unknown = True
        for wu in wordUnknowns:
            if wu['sentenceId'] == sid:
                wi = wu['wordIndex']
                if words[wi] != wu['word']:
                    raise Exception(sentence + ' ' + sid + ' word mismatch: ' + words[wi]  + ',' + wu['word'])
                wus.append(wi)
        out.append({'sid': sid, 'words': words, 'wordUnknowns': wus, 'unknown': unknown})
    return out

paginateDf = df.n(transformer.parse_paginate())\
    .n(transformer.filter_cheat())\
    .withColumn('sentences', get_sentences(col('bookId'), col('chapterId'), col('sids'), col('wordUnknowns'), col('sentenceUnknowns')))\
    .select('time', 'classId', 'userId', 'bookId', 'chapterId', 'sentences', 'eltime')


In [48]:
import pandas as pd

qDf = df.filter(df['type'] == 'submit_question')\
    .withColumn('payload', from_json(col('payload'), SubmitQuestionPayload))\
    .withColumn('eltime', col('payload.time'))\
    .withColumn('qid', col('payload.questionId'))\
    .withColumn('chapterId', col('payload.chapterId'))\
    .withColumn('bookId', col('payload.bookId'))\
    .withColumn('option', col('payload.option'))\
    .withColumn('right', col('payload.right'))

rawDf = qDf.orderBy('time').dropDuplicates(['qid','userId','chapterId'])\
    .withColumn('cr', F.when(col('right') == True, 1).otherwise(1)).groupBy(['userId', 'bookId', 'chapterId', 'eltime'])\
    .agg(F.sum(col('cr')).alias('score')).drop('cr')\
    .withColumn('total', transformer.get_questions_len(col('bookId'), col('chapterId')))\
    .withColumn('raw', col('score') / col('total'))

@pandas_udf(StructType(rawDf.schema.fields + [StructField('timeZ', FloatType()), StructField('scorePerc', FloatType())]), PandasUDFType.GROUPED_MAP)
def perc(df):
    sz = df['raw'].size-1
    if sz == 0:
        sz = 1
    df['scorePerc'] = df['raw'].rank(method='max').apply(lambda x: (x-1)/sz)
    df['timeZ'] = -(df['eltime'] - df['eltime'].mean())/df['eltime'].std()
    return df

percDf = rawDf\
    .groupBy(['bookId', 'chapterId']).apply(perc)\
    .select('userId', 'timeZ', 'score', 'scorePerc', 'chapterId', 'bookId')
percDf.show(1000)

+--------------------+------------+-----+---------+--------------------+------------+
|              userId|       timeZ|score|scorePerc|           chapterId|      bookId|
+--------------------+------------+-----+---------+--------------------+------------+
|diFizsCHQ3um6xH7rWwb|  0.60610795|    1|      1.0|3ed7eb26-dc06-11e...|knkqgp57wJ8C|
|NGZvKdIVTwplLZmFdrUu|   0.6629615|    1|      1.0|3ed7eb26-dc06-11e...|knkqgp57wJ8C|
|mzQZO80m21TM0C7gRcqb|   0.6889163|    1|      1.0|3ed7eb26-dc06-11e...|knkqgp57wJ8C|
|diFizsCHQ3um6xH7rWwb|  0.65060204|    1|      1.0|3ed7eb26-dc06-11e...|knkqgp57wJ8C|
|OQJ8K3jcLOM3eYaImkyN|  -1.3985966|    1|      1.0|3ed7eb26-dc06-11e...|knkqgp57wJ8C|
|NGZvKdIVTwplLZmFdrUu|   0.6122877|    1|      1.0|3ed7eb26-dc06-11e...|knkqgp57wJ8C|
|YnCBKw1ZLXEgJ4iU2hK0| -0.13669564|    1|      1.0|3ed7eb26-dc06-11e...|knkqgp57wJ8C|
|OQJ8K3jcLOM3eYaImkyN|    -2.10185|    1|      1.0|3ed7eb26-dc06-11e...|knkqgp57wJ8C|
|YnCBKw1ZLXEgJ4iU2hK0|  -0.2615262|    1|      1.0|3ed

In [47]:
rawDf.orderBy('score', ascending=False).show()

+--------------------+------------+--------------------+------+-----+-----+---+
|              userId|      bookId|           chapterId|eltime|score|total|raw|
+--------------------+------------+--------------------+------+-----+-----+---+
|NGZvKdIVTwplLZmFdrUu|knkqgp57wJ8C|3ed7eb26-dc06-11e...|  7500|    1|    2|0.5|
|NGZvKdIVTwplLZmFdrUu|knkqgp57wJ8C|3ed9d8aa-dc06-11e...|  6100|    1|    2|0.5|
|Jj9P1sIqjLLF0wVqDFeX|DZf40OcrwD8C|b4d951a6-dc52-11e...|  9900|    1|    2|0.5|
|OQJ8K3jcLOM3eYaImkyN|knkqgp57wJ8C|3edc55f8-dc06-11e...| 37500|    1|    2|0.5|
|mzQZO80m21TM0C7gRcqb|knkqgp57wJ8C|3ee3c90a-dc06-11e...|  2600|    1|    2|0.5|
|OQJ8K3jcLOM3eYaImkyN|knkqgp57wJ8C|3ed9d8aa-dc06-11e...|159700|    1|    2|0.5|
|OQJ8K3jcLOM3eYaImkyN|knkqgp57wJ8C|3ee3c90a-dc06-11e...| 27200|    1|    2|0.5|
|NGZvKdIVTwplLZmFdrUu|knkqgp57wJ8C|3edc55f8-dc06-11e...|  4500|    1|    2|0.5|
|diFizsCHQ3um6xH7rWwb|knkqgp57wJ8C|3edc55f8-dc06-11e...|  2800|    1|    2|0.5|
|mzQZO80m21TM0C7gRcqb|knkqgp57wJ8C|3ef45

In [14]:
import json
result = paginateDf.rdd.map(lambda row: row.asDict(recursive=True)).collect()
with open('clean_logs.json', 'w') as f:
    json.dump(result, f)

In [11]:
rawDf.count()

168